## In this tutorial we will be using pretrained model on out dataset

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
sz = 224
batch_size = 64

In [3]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.layers import Dropout,Flatten,Dense
from keras.applications import ResNet50
from keras.models import Model,Sequential
from keras.layers import Dense,GlobalAveragePooling2D
import keras as K
from keras.applications.resnet50 import preprocess_input

/home/paperspace/anaconda3/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Setting training and validation directory

In [4]:
train_data_dir = 'data/dogscats/train/'
validation_data_dir = 'data/dogscats/valid/'


In [5]:
train_datagen = ImageDataGenerator(
                preprocessing_function=preprocess_input,
                shear_range=0.2,
                zoom_range = 0.2,
                horizontal_flip = True)

test_datagen = ImageDataGenerator(
                preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size = (sz,sz),
                                                   batch_size = batch_size,class_mode = 'binary')

validation_generator = test_datagen.flow_from_directory(validation_data_dir,
                                                       shuffle = False,
                                                       target_size = (sz,sz),
                                                       batch_size = batch_size,
                                                       class_mode = 'binary')

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


We are modifying last layer of the model to give give output as per our need
0 means it is a cat and 1 means dog

In [6]:
base_model = ResNet50(weights = 'imagenet',include_top = False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation='relu')(x)
prediction = Dense(1,activation='sigmoid')(x)

In [8]:
model = Model(inputs=base_model.inputs,output = prediction)
for layer in base_model.layers:layer.trainable = False
model.compile(optimizer = 'rmsprop',loss = 'binary_crossentropy',metrics = ['accuracy'])

/home/paperspace/anaconda3/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=Tensor("de...)`
  """Entry point for launching an IPython kernel.


In [11]:
%%time
model.fit_generator(train_generator,
                    train_generator.n // batch_size,
                   epochs = 3,
                   workers = 4,
                   validation_data = validation_generator,
                   validation_steps = validation_generator.n // batch_size)


Epoch 1/3
359/359 [==============================] - 119s 330ms/step - loss: 0.1926 - acc: 0.9479 - val_loss: 0.0667 - val_acc: 0.9829
Epoch 2/3
359/359 [==============================] - 107s 299ms/step - loss: 0.0864 - acc: 0.9700 - val_loss: 0.0706 - val_acc: 0.9808
Epoch 3/3
359/359 [==============================] - 107s 297ms/step - loss: 0.0634 - acc: 0.9778 - val_loss: 0.0796 - val_acc: 0.9829
CPU times: user 20min 11s, sys: 30.5 s, total: 20min 42s
Wall time: 5min 32s


## Spliting model to train only last few layers

In [13]:
split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
model.compile(optimizer = 'rmsprop',loss ='binary_crossentropy',metrics=['accuracy'])


In [18]:
%%time
model.fit_generator(train_generator,
                    train_generator.n // batch_size,
                    epochs = 1,workers = 3,
                   validation_data = validation_generator,
                   validation_steps = validation_generator.n // batch_size)

Epoch 1/1
359/359 [==============================] - 134s 374ms/step - loss: 0.0378 - acc: 0.9865 - val_loss: 0.0979 - val_acc: 0.9839
CPU times: user 7min 9s, sys: 12.8 s, total: 7min 21s
Wall time: 2min 14s
